![image_1768962450396.png](./image_1768962450396.png "image_1768962450396.png")

1.1 Peek at the data (row counts)
-- How many rows do we have?

In [0]:
SELECT 'device_messages_raw' AS table_name, COUNT(*) AS rows FROM workspace.bronze.device_messages_raw

UNION ALL

SELECT 'rapid_step_tests_raw', COUNT(*) FROM workspace.bronze.rapid_step_tests_raw;


![image_1769031381769.png](./image_1769031381769.png "image_1769031381769.png")

In [0]:
-- Count messages per device (top 10)

SELECT deviceId, COUNT(*) AS messages

FROM workspace.bronze.device_messages_raw

GROUP BY deviceId

ORDER BY messages DESC

LIMIT 10;



![image_1769031415525.png](./image_1769031415525.png "image_1769031415525.png")

In [0]:
-- Create a cleaned view for convenience during lab (session-scoped)

CREATE OR REPLACE TEMP VIEW device_messages_clean AS

SELECT

  device_id,

  TRY_CAST(REGEXP_REPLACE(CAST(distance AS STRING), '[^0-9.]', '') AS DOUBLE) AS distance_cm,

  CAST(timestamp AS BIGINT) AS ts_ms,

  CAST(date AS BIGINT)       AS date_ms,

  message_origin,

  sensor_type,

  message

FROM workspace.bronze.device_messages_raw;

-- Check the cleaning

SELECT * FROM device_messages_clean LIMIT 20;



![image_1769031483523.png](./image_1769031483523.png "image_1769031483523.png")

In [0]:
SELECT device_id,

       AVG(distance_cm) AS avg_cm,

       MIN(distance_cm) AS min_cm,

       MAX(distance_cm) AS max_cm

FROM device_messages_clean

GROUP BY device_id

ORDER BY avg_cm DESC

LIMIT 10;



![image_1769031559424.png](./image_1769031559424.png "image_1769031559424.png")

In [0]:
-- Average/min/max distance per device (first 10 devices)

SELECT device_id,

       AVG(distance_cm) AS avg_cm,

       MIN(distance_cm) AS min_cm,

       MAX(distance_cm) AS max_cm

FROM device_messages_clean

GROUP BY device_id

ORDER BY avg_cm DESC

LIMIT 10;


![image_1769031596214.png](./image_1769031596214.png "image_1769031596214.png")

In [0]:
-- One row per step interval

WITH exploded AS (

  SELECT

    customer,

    device_id,

    start_time,

    posexplode(step_points) AS (step_index, step_ms)

  FROM workspace.bronze.rapid_step_tests_raw

)

SELECT customer, device_id,

       COUNT(*) AS steps,

       AVG(step_ms) AS avg_step_ms,

       STDDEV(step_ms) AS sd_step_ms

FROM exploded

GROUP BY customer, device_id

ORDER BY steps DESC

LIMIT 10;


![image_1769031748678.png](./image_1769031748678.png "image_1769031748678.png")

In [0]:

WITH tests AS (

  SELECT customer, device_id, start_time, stop_time, test_time, total_steps

  FROM workspace.bronze.rapid_step_tests_raw

),

msgs AS (

  SELECT deviceId, ts_ms, distance_cm

  FROM device_messages_clean

  WHERE distance_cm IS NOT NULL

)

SELECT

  t.customer, t.device_id, t.start_time, t.stop_time,

  COUNT(m.ts_ms)                   AS readings_in_window,

  AVG(m.distance_cm)               AS avg_cm_in_window,

  MIN(m.distance_cm)               AS min_cm_in_window,

  MAX(m.distance_cm)               AS max_cm_in_window

FROM tests t

JOIN msgs  m

  ON m.deviceId = t.device_id

 AND m.ts_ms BETWEEN t.start_time AND t.stop_time

GROUP BY t.customer, t.device_id, t.start_time, t.stop_time

ORDER BY readings_in_window DESC

LIMIT 20;


![image_1769031853645.png](./image_1769031853645.png "image_1769031853645.png")